In [1]:
import pandas as pd
import datetime
import numpy as np
import os.path
import wget
from zipfile import ZipFile
import shutil
import re
from multiprocessing import Pool
import multiprocessing as mp

In [2]:
## Variaveis globais

#qual base de dados quero usar, "con" para consolidado e "ind" para individual
tipo = "con"

#Pega data corrente e ano corrente
date = datetime.date.today()
year = date.year

# ano inicial pra coleta dos dados
year_0 = 2010 
year_0_tri = year-2 # ano inicial pra coletoa dos dados trimestrais

In [24]:
# baixa todos os dados se não estiverem presentes, retorna uma lista de nomes de arquivos baixados
def baixa_dados(year_0=year_0, year=year, year_0_tri=year_0_tri):
    # URL's onde estão os arquivos dfp e itr na CVM
    url_base = 'http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/'
    url_base_fre = 'http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/FRE/DADOS/'
    url_base_tri = 'http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/'
    url_base_fca = "https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/FCA/DADOS/"

    # Cria uma lista com os nomes dos arquivos de dados anuais (dfp) da CVM
    arquivos_zip = []
    for ano in range(year_0, year + 1):
        arquivos_zip.append(f'dfp_cia_aberta_{ano}.zip')

    # baixa os arquivos dfp
    arquivos_zip_dfp_baixados = []
    for arq in arquivos_zip:
        if (os.path.isfile(f'./Data/{arq}')):
            print('%s já esta aqui' % arq)
            arquivos_zip_dfp_baixados.append(arq)
        else:
            try:
                wget.download(url_base + arq, out=f"./Data/{arq}")
                arquivos_zip_dfp_baixados.append(arq)
            except:
                print(f"falha ao baixar {arq}")
                pass

    # Cria lista de arquivos ITR (trimestrais) pra baixar
    arquivos_zip = []
    for ano in range(year_0_tri, year + 1):
        arquivos_zip.append(f'itr_cia_aberta_{ano}.zip')

    # baixa os arquivos ITR (dados trimestrais) se não estiverem presentes
    arquivos_zip_itr_baixados = []
    for arq in arquivos_zip:
        if (os.path.isfile(f'./Data/{arq}')):
            print('%s já esta aqui' % arq)
            arquivos_zip_itr_baixados.append(arq)
        else:
            try:
                wget.download(url_base_tri + arq, out=f"./Data/{arq}")
                arquivos_zip_itr_baixados.append(arq)
            except:
                print(f"falha ao baixar {arq}")
                pass

    # baixa arquivo FCA, que contém os tickers

    # Cria lista de arquivos FCA  pra baixar. Este aqui só precisa baixar o do último ano disponível
    # Se não tiver do ano atual baixa o do ano passado
    arquivos_zip = []
    for ano in [year, year - 1]:
        arquivos_zip.append(f'fca_cia_aberta_{ano}.zip')

    arquivos_zip_fca_baixados = []
    for arq in arquivos_zip:
        if (os.path.isfile(f'./Data/{arq}')):
            print('%s já esta aqui' % arq)
            arquivos_zip_fca_baixados.append(arq)
            break
        else:
            try:
                wget.download(url_base_fca + arq, out=f"./Data/{arq}")
                arquivos_zip_fca_baixados.append(arq)
                break # se conseguiu baixar 1 já sai do loop
            except:
                print(f"falha ao baixar {arq}")
    
    # baixa arquivo FRE, que contém o número total de ações
    
    arquivos_zip = []
    for ano in range(year-3, year+1):
        print(ano)
        arquivos_zip.append(f'fre_cia_aberta_{ano}.zip')

    arquivos_zip_fre_baixados = []
    for arq in arquivos_zip:
        if (os.path.isfile(f'./Data/{arq}')):
            print('%s já esta aqui' % arq)
            arquivos_zip_fre_baixados.append(arq)
        else:
            try:
                wget.download(url_base_fre + arq, out=f"./Data/{arq}")
                arquivos_zip_fre_baixados.append(arq)
            except:
                print(f"falha ao baixar {arq}")
                
    return (arquivos_zip_dfp_baixados, arquivos_zip_itr_baixados, arquivos_zip_fca_baixados, arquivos_zip_fre_baixados)

In [53]:
def processa_dados(year_0=year_0, year=year, year_0_tri=year_0_tri):
    print("Processando dados")
    # junta todos os anos de dados anuais em um arquivo só de cada tipo
    nomes = ['BPA_con', 'BPA_ind', 'BPP_con', 'BPP_ind', 'DRE_con', 'DRE_ind',
             'DFC_MD_con', 'DFC_MI_con', 'DFC_MD_ind', 'DFC_MI_ind']
    for nome in nomes:
        arquivo_bp = pd.DataFrame()
        for ano in range(year_0, year):
            try:
                arquivo_bp = pd.concat([arquivo_bp,
                                        pd.read_csv(f'./Data/temp/dfp_cia_aberta_{nome}_{ano}.csv', sep=';',
                                                    decimal=',', encoding='ISO-8859-1')])
            except:
                print(f"falha em {nome}")
                pass
        arquivo_bp.to_csv(f'./Data/processados/dfp_cia_aberta_{nome}_{year_0}-{year}.csv', index=False, encoding="latin1")
        print(f'DFP {nome} processado')

    # junta todos os anos de dados trimestrais em um arquivo só de cada tipo
    nomes_tri = ['BPA_con', 'BPA_ind', 'BPP_con', 'BPP_ind', 'DRE_con', 'DRE_ind',
                 "DFC_MD_con", "DFC_MI_con", "DFC_MD_ind", "DFC_MI_ind"]
    for nome in nomes_tri:
        arquivo_bp = pd.DataFrame()
        for ano in range(year_0_tri, year + 1):
            try:
                arquivo_bp = pd.concat([arquivo_bp,
                                        pd.read_csv(f'./Data/temp/itr_cia_aberta_{nome}_{ano}.csv', sep=';',
                                                    decimal=',', encoding='ISO-8859-1')])
            except:
                print(f"falha em itr_cia_aberta_{nome}_{ano}")
                pass
        arquivo_bp.to_csv(f'./Data/processados/itr_cia_aberta_{nome}_{year_0_tri}-{year}.csv', index=False, encoding="latin1")
        print(f'ITR {nome} processado')
    
    # cria uma lista com todos os arquivos fre
    arq_fre = [item for item in os.listdir("./Data/temp") if item.startswith("fre_cia_aberta_distribuicao_capital_2")]
    fre = pd.DataFrame()

    # concatena tudo
    for arq in arq_fre[::-1]:
        fre = pd.concat([fre, pd.read_csv(f"./Data/temp/{arq}", sep=";")], axis=0)

    # joga fora as linhas com 0
    fre = fre[fre["Quantidade_Total_Acoes_Circulacao"] != 0]

    # ordena pelo mais recente, retira as duplicatas e pega só as colunas que interessam
    fre = fre.sort_values(by=["Data_Referencia", "Versao"], ascending=False).drop_duplicates(["CNPJ_Companhia"], keep="first", ignore_index=True)[["CNPJ_Companhia", "Quantidade_Total_Acoes_Circulacao"]]

    # salva em um csv
    fre.to_csv("./Data/processados/qtd_acoes.csv", encoding="latin1", index=False)
    fre.set_index("CNPJ_Companhia", inplace=True)
    print("FRE processado")
    
        
    print("Arquivos processados")

In [6]:
# só extrai o arquivo dado como parâmetro para a pasta ./Data/temp
def extrai_arquivo(arq):
    try:
        ZipFile(f"./Data/{arq}", 'r').extractall('./Data/temp')
        print(f"{arq} extraido")
    except:
        print(f"erro ao extrair {arq}")

# extrai uma lista de arquivos com paralelismo. não roda no jupyter

def extrai_paralelo(arquivos_para_extrair):

    #  descobre quantas threads tem o CPU atual e cria uma pool para extrair os dados com paralelismo
    procs = mp.cpu_count()
    pool = Pool(procs)

    # cria uma lista de pools para depois dar get
    lista_pool = []
    for arq in arquivos_para_extrair:
        lista_pool.append(pool.apply_async(extrai_arquivo, args=(arq,)))
    print("Extraindo")
    output = [item.get() for item in lista_pool]
    return(output)

In [7]:
# faz o usuario escolher uma empresa
def escolhe_empresa():
    lista_empresas = \
    pd.read_csv(f"./Data/processados/dfp_cia_aberta_BPA_{tipo}_{year_0}-{year}.csv", header=0, sep=",")[
        ["DENOM_CIA", 'CD_CVM']].drop_duplicates()
    while True:
        busca = input("Entre com o nome da empresa: ")
        empresa = lista_empresas[lista_empresas['DENOM_CIA'].str.contains(busca, case=False)]
        print(empresa)
        ok = input("A empresa desejada está na primeira linha? (y/n) ")
        if ok == "y":
            return(busca)

In [54]:
# retorna um df com CNPJ razao social e ticker de todas as empresas listadas em bolsa
def cria_lista_empresas(year_0=year_0, year=year, year_0_tri=year_0_tri):

    #cria lista de empresas. Concatena consolidado e individual
    lista_empresas = pd.read_csv(f"./Data/processados/dfp_cia_aberta_BPA_con_{year_0}-{year}.csv", header=0, sep=",", )[["DENOM_CIA", 'CD_CVM', "CNPJ_CIA"]].drop_duplicates()
    lista_empresas = pd.concat([lista_empresas, pd.read_csv(f"./Data/processados/dfp_cia_aberta_BPA_ind_{year_0}-{year}.csv", header=0, sep=",", )[["DENOM_CIA", 'CD_CVM', "CNPJ_CIA"]].drop_duplicates()], ignore_index=True).drop_duplicates(keep="first")
    lista_empresas.set_index("CNPJ_CIA", inplace=True)

    # coloca tickers do arquivo fca
    lista_empresas = lista_empresas.join(pd.read_csv(f"./Data/processados/{arq_fca}", header=0, sep=";", encoding="latin1").set_index("CNPJ_Companhia")["Codigo_Negociacao"].dropna())

    # pega tickers da B3 tbm pq na CVM é incompleto.
    # extraí de https://www.b3.com.br/pt_br/produtos-e-servicos/negociacao/renda-variavel/empresas-listadas.htm
    # no futuro seria bom automatizar isso com um webscraper
    b3 = pd.read_csv("./Data/empresas_listadas_b3.csv", encoding="latin1", sep=";")[["Razão Social","Código"]]

    # Cria a coluna CNPJ com os dados do index
    lista_empresas = lista_empresas.rename_axis('CNPJ').reset_index()

    # Junta os dados da b3 no arquivo original
    lista_empresas = lista_empresas.merge(b3, left_on="DENOM_CIA", right_on="Razão Social", how="left").drop(columns=["Razão Social"])
    
    # Dá replace em todas as linhas de "Codigo_Negociacao" que não obedecem o padrão "ABCD1" ou "ABCD11"
    lista_empresas["Codigo_Negociacao"].replace(to_replace= r"^(?!\D{4}\d{1,2}$).*", value= np.nan, regex=True, inplace=True)

    # Pega só as 4 primeiras letras da coluna "Codigo_Negociacao"
    lista_empresas["TICKER"] = lista_empresas["Codigo_Negociacao"].str[:4]
    lista_empresas.drop(columns=["Codigo_Negociacao"], inplace=True)
    lista_empresas.drop_duplicates(inplace=True)

    # Dá replace nos NaN da CVM com o ticker do site da B3
    lista_empresas["TICKER"].fillna(lista_empresas["Código"], inplace=True)

    # Joga fora a coluna com tickers da B3
    lista_empresas.drop(columns=["Código"], inplace=True)
    
    # joga fora o que não tiver ticker
    lista_empresas.dropna(inplace=True)
    
    #deixa o index como o CNPJ
    lista_empresas.set_index("CNPJ")
    
    # tira linhas duplicadas (tem que ter CNPJ, CD_CVM e TICKER iguais)
    lista_empresas.drop_duplicates(["CNPJ", "CD_CVM", "TICKER"], inplace=True)
    
    #salva em um csv
    lista_empresas.to_csv("./Data/processados/lista empresas.csv", encoding="latin1", index=False)
    
    return(lista_empresas)

## Main

In [9]:
# Limpeza

# Deleta o conteúdo da pasta processados se existir
lista_arquivos = os.listdir("./Data/processados/")
lista_arquivos.remove(".gitignore")

for item in lista_arquivos:
    try:
        os.remove(f"./Data/processados/{item}")
    except:
        pass

#deleta pasta temporaria se existir
try:
    shutil.rmtree(f'./Data/temp')
except:
    pass

In [21]:
# baixa os dados e coloca os nomes dos arquivos baixados em cada variavel
arquivos_dfp, arquivos_itr, arquivos_fca, arquivos_fre = baixa_dados(year_0, year, year_0_tri)
arquivos_dfp, arquivos_itr, arquivos_fca, arquivos_fre

dfp_cia_aberta_2010.zip já esta aqui
dfp_cia_aberta_2011.zip já esta aqui
dfp_cia_aberta_2012.zip já esta aqui
dfp_cia_aberta_2013.zip já esta aqui
dfp_cia_aberta_2014.zip já esta aqui
dfp_cia_aberta_2015.zip já esta aqui
dfp_cia_aberta_2016.zip já esta aqui
dfp_cia_aberta_2017.zip já esta aqui
dfp_cia_aberta_2018.zip já esta aqui
dfp_cia_aberta_2019.zip já esta aqui
dfp_cia_aberta_2020.zip já esta aqui
dfp_cia_aberta_2021.zip já esta aqui
dfp_cia_aberta_2022.zip já esta aqui
itr_cia_aberta_2020.zip já esta aqui
itr_cia_aberta_2021.zip já esta aqui
itr_cia_aberta_2022.zip já esta aqui
fca_cia_aberta_2022.zip já esta aqui


(['dfp_cia_aberta_2010.zip',
  'dfp_cia_aberta_2011.zip',
  'dfp_cia_aberta_2012.zip',
  'dfp_cia_aberta_2013.zip',
  'dfp_cia_aberta_2014.zip',
  'dfp_cia_aberta_2015.zip',
  'dfp_cia_aberta_2016.zip',
  'dfp_cia_aberta_2017.zip',
  'dfp_cia_aberta_2018.zip',
  'dfp_cia_aberta_2019.zip',
  'dfp_cia_aberta_2020.zip',
  'dfp_cia_aberta_2021.zip',
  'dfp_cia_aberta_2022.zip'],
 ['itr_cia_aberta_2020.zip',
  'itr_cia_aberta_2021.zip',
  'itr_cia_aberta_2022.zip'],
 ['fca_cia_aberta_2022.zip'],
 [])

In [15]:
# extrai arquivos
arquivos_para_extrair = arquivos_dfp + arquivos_itr + arquivos_fca + arquivos_fre

# extrai sequencial
for arq in arquivos_para_extrair:
    extrai_arquivo(arq)

# extrai paralelo, não funciona no jupyter
#extrai_paralelo(arquivos_para_extrair)

dfp_cia_aberta_2010.zip extraido
dfp_cia_aberta_2011.zip extraido
dfp_cia_aberta_2012.zip extraido
dfp_cia_aberta_2013.zip extraido
dfp_cia_aberta_2014.zip extraido
dfp_cia_aberta_2015.zip extraido
dfp_cia_aberta_2016.zip extraido
dfp_cia_aberta_2017.zip extraido
dfp_cia_aberta_2018.zip extraido
dfp_cia_aberta_2019.zip extraido
dfp_cia_aberta_2020.zip extraido
dfp_cia_aberta_2021.zip extraido
dfp_cia_aberta_2022.zip extraido
itr_cia_aberta_2020.zip extraido
itr_cia_aberta_2021.zip extraido
itr_cia_aberta_2022.zip extraido
fca_cia_aberta_2022.zip extraido
fre_cia_aberta_2019.zip extraido
fre_cia_aberta_2020.zip extraido
fre_cia_aberta_2021.zip extraido
fre_cia_aberta_2022.zip extraido


In [12]:
# pega o nome do arquivo fca que contem os tickers
arq_fca = [item for item in os.listdir("./Data/temp") if item.startswith("fca_cia_aberta_valor_mobiliario")]
arq_fca = arq_fca[0]

# copia o arquivo fca pra pasta processados
shutil.copyfile(f"./Data/temp/{arq_fca}", f"./Data/processados/{arq_fca}")

'./Data/processados/fca_cia_aberta_valor_mobiliario_2022.csv'

In [13]:
# concatena os arquivos baixados do ano inicial até o final
processa_dados(year_0, year, year_0_tri)

#deleta pasta temporaria se existir
try:
    shutil.rmtree(f'./Data/temp')
except:
    pass

Processando dados
DFP BPA_con processado
DFP BPA_ind processado
DFP BPP_con processado
DFP BPP_ind processado
DFP DRE_con processado
DFP DRE_ind processado
DFP DFC_MD_con processado
DFP DFC_MI_con processado
DFP DFC_MD_ind processado
DFP DFC_MI_ind processado
ITR BPA_con processado
ITR BPA_ind processado
ITR BPP_con processado
ITR BPP_ind processado
ITR DRE_con processado
ITR DRE_ind processado
ITR DFC_MD_con processado
ITR DFC_MI_con processado
ITR DFC_MD_ind processado
ITR DFC_MI_ind processado
Arquivos processados


In [60]:
# cria um dataframe com todas as empresas listadas e ticker
lista_empresas = cria_lista_empresas(year_0, year, year_0_tri)

# muda o index pro CNPJ pra poder dar join
lista_empresas.set_index("CNPJ", inplace=True)

# coloca a quantidade de ações em circulação e tira as que não tem
lista_empresas = lista_empresas.join(fre).dropna()
lista_empresas

,DENOM_CIA,CD_CVM,TICKER,Quantidade_Total_Acoes_Circulacao
00.000.000/0001-91,BCO BRASIL S.A.,1023,BBAS,1.420731e+09
00.000.208/0001-00,BRB BCO DE BRASILIA S.A.,14206,BSLI,4.195216e+07
00.001.180/0001-26,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,ELET,5.997421e+08
00.070.698/0001-11,CIA ENERGETICA DE BRASILIA,14451,CEBR,1.430870e+07
00.242.184/0001-04,"ARMAC LOCAÇÃO, LOGÍSTICA E SERVIÇOS S.A.",26069,ARML,1.718094e+08
...,...,...,...,...
94.813.102/0001-70,TRÊS TENTOS AGROINDUSTRIAL S/A,25950,TTEN,1.123436e+08
95.426.862/0001-97,EXCELSIOR ALIMENTOS S.A.,1570,BAUH,1.508636e+06
96.418.264/0218-02,LOJAS QUERO QUERO S.A.,25038,LJQQ,1.789674e+08
97.191.902/0001-94,CONSERVAS ODERICH S.A.,4693,ODER,1.979520e+05


In [58]:
# calcular indicadores e colocar cotação (adiar ao maximo pq demora mto tempo pra pegar)